## Exploring SCAN

Each example in the SCAN dataset is aimed at converting a natural language command to a sequence of actions. 

$$ InputCommand \longrightarrow OutputSequence$$

Example: 

$$jump \qquad  thrice \longrightarrow JUMP \qquad  JUMP\qquad  JUMP$$

#### Interpreting Input Command

The input command is itself understood compositionally and can be generated with a basic PS grammar. The nodes involve: 

1. C $\longrightarrow$ S and S
2. C $\longrightarrow$ S after S
3. C $\longrightarrow$ S
4. S $\longrightarrow$ V twice
5. S $\longrightarrow$ V thrice
6. S $\longrightarrow$ V
7. V $\longrightarrow$ D[1] opposite D[2]
8. V $\longrightarrow$ D[1] around D[2]
9. V $\longrightarrow$ D
10. V $\longrightarrow$ U
11. D $\longrightarrow$ U left
12. D $\longrightarrow$ U right
13. D $\longrightarrow$ turn left
14. D $\longrightarrow$ turn right
15. U $\longrightarrow$ walk
16. U $\longrightarrow$ run
17. U $\longrightarrow$ jump
18. U $\longrightarrow$ look

Where C=Full Command, S= Sentence Phrase, V= Verb Phrase, D= Direction Phrase, U= Verb

In [3]:
# defining a class Node
# it is used to represent nodes in a composition tree for a given command
# calling Node(command) will initialize root of a composition tree for that command
class Node:
    # create new instance of class with input command
    def __init__(self, command): 
        self.command = command # assigns command string to instance variable
        self.children = [] # creates an empty list of children nodes
    
    # function to create child nodes
    def add_child(self, child): 
        self.children.append(child)
    
    # function to define how node instance is described as a string
    def __repr__(self, level=0):
        ret = "\t" * level + repr(self.command) + "\n"
        for child in self.children:
            ret += child.__repr__(level + 1) # loops through children to increase indent level
        return ret

In [4]:
# function to build composition tree using a command
def build_composition_tree(command):
    # for one command simple decomposition
    # to be replaced with phrase structure grammar
    decompositions = {
        "jump around left": ["jump around", "left"],
        "jump around": ["jump", "around"],
        "around left": ["around", "left"],
        "jump": [],
        "around": [],
        "left": [],
    }
    # initiating tree structure with decompositions acting as children nodes
    root = Node(command)
    if command in decompositions:
        for sub_command in decompositions[command]:
            root.add_child(build_composition_tree(sub_command))
    return root

# Example command from SCAN dataset
command = "jump around left"
composition_tree = build_composition_tree(command)
print(composition_tree)

'jump around left'
	'jump around'
		'jump'
		'around'
	'left'



#### Mapping Input Command to Output Sequence

The output action sequence is created by the interpretation function which takes in elements at each child node and 

In [5]:
def interpret_command(command):
    action_sequences = {
    "jump": ["JUMP"],
    "walk": ["WALK"],
    "look": ["LOOK"],
    "run": ["RUN"],
    "turn left": ["LTURN"],
    "turn right": ["RTURN"],
    "around left": ["LTURN", "LTURN", "LTURN", "LTURN"],
    "around right": ["RTURN", "RTURN", "RTURN", "RTURN"]
    }
    decompositions = {
        "jump around left": ["jump", "around left"],
        "jump around right": ["jump", "around right"],
        "walk around left": ["walk", "around left"],
        "walk around right": ["walk", "around right"],
        "jump twice": ["jump", "jump"],
        "walk twice": ["walk", "walk"],
        "jump thrice": ["jump", "jump", "jump"],
        "walk thrice": ["walk", "walk", "walk"],
        "jump around left twice": ["jump around left", "jump around left"],
        "jump around left thrice": ["jump around left", "jump around left", "jump around left"],
        "jump around right twice": ["jump around right", "jump around right"],
        "jump around right thrice": ["jump around right", "jump around right", "jump around right"],
        "jump and walk": ["jump", "walk"],
        "walk and jump": ["walk", "jump"],
        "jump after walk": ["walk", "jump"],
        "walk after jump": ["jump", "walk"]
    }
    
    if command in action_sequences:
        return action_sequences[command]
    
    if command in decompositions:
        result = []
        for sub_command in decompositions[command]:
            result.extend(interpret_command(sub_command))
        return result
    
    return []

# Example command from SCAN dataset
command = "jump around left twice"
action_sequence = interpret_command(command)
print(action_sequence)

['JUMP', 'LTURN', 'LTURN', 'LTURN', 'LTURN', 'JUMP', 'LTURN', 'LTURN', 'LTURN', 'LTURN']


## Building Causal Model